In [1]:
import pandas as pd
# %%capture
%pip install datasets evaluate install transformers[torch]  nltk bitsandbytes rouge_score
# %pip install rouge_score
import nltk
import bitsandbytes
nltk.download("punkt")
from nltk.tokenize import sent_tokenize
from datasets import load_dataset
import datasets
from transformers import AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
import numpy as np
import evaluate
import argparse
#obviously, secret_token.py is not included in this repository (in gitignore)
# from secret_token import token
import torch
import torch.nn.functional as F
from sklearn.utils.class_weight import compute_class_weight
# from clean_utils import get_special_tokens
from huggingface_hub import notebook_login

notebook_login()
# from clean_utils import get_special_tokens

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 35.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [28]:
# def get_special_tokens():
#     #[SEP] is already a special token in BERT, these are additional special tokens
#     return ['[USER]', '[LINK]', '[EMAIL]', '[IMAGE]', '[QUOTED_TEXT]', '[EMPTY]', '[HTML]']

In [62]:
dataset = load_dataset("webis/tldr-17", split="train")

Loading dataset shards:   0%|          | 0/38 [00:00<?, ?it/s]

In [16]:
dataset = dataset.train_test_split(test_size=0.05)
# print(dataset.column_names)
# print(dataset.column_names)
# random_sample["train"] = dataset["train"].shuffle(seed=42).select(range(60000))
# random_train, random_test = dataset.train_test_split(test_size=0.05)
# print(random_sample)
# random_sample, remaining_sample = dataset["train"].random_split([60000, len(dataset)-60000])
# random_sample = random_sample.train_test_split(test_size=0.05)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

In [19]:
random_train = dataset["train"].shuffle(seed=42).select(range(57000))
random_test = dataset["test"].shuffle(seed=42).select(range(3000))

In [20]:
def tokenize_function(examples):
        # print(examples)
        model_inputs = tokenizer(examples["content"], padding = "max_length", max_length = 1024, truncation=True)
        summary = tokenizer(examples["summary"], padding = "max_length", max_length=128, truncation=True)
        model_inputs["labels"] = summary["input_ids"]
        return model_inputs

In [46]:
tokenized_train = random_train.map(tokenize_function, batched=True)
tokenized_test = random_test.map(tokenize_function, batched=True)

Map:   0%|          | 0/57000 [00:00<?, ? examples/s]

Map:   0%|          | 0/3000 [00:00<?, ? examples/s]

In [39]:

# train_dataset = tokenized_train.shuffle(seed=42).select(range(1000))
# eval_dataset = tokenized_test.shuffle(seed=42).select(range(80))
train_dataset = tokenized_train
eval_dataset = tokenized_test
# train_dataset = tokenized_dataset["train"]
# eval_dataset = tokenized_dataset["test"]

In [23]:
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [44]:
batch_size = 8
num_train_epochs = 3

logging_steps = len(train_dataset) // batch_size
args = Seq2SeqTrainingArguments(
    output_dir=f"model-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=num_train_epochs,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=False,
    fp16 = True,
    optim="adamw_bnb_8bit"
)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    # Decode generated summaries into text
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Decode reference summaries into text
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # ROUGE expects a newline after each sentence
    decoded_preds = ["\n".join(sent_tokenize(pred.strip())) for pred in decoded_preds]
    print(len(decoded_preds))
    decoded_labels = ["\n".join(sent_tokenize(label.strip())) for label in decoded_labels]
    print(len(decoded_labels))
    # Compute ROUGE scores
    rouge_score = evaluate.load("rouge")
    result = rouge_score.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )
    print(result)
    # Extract the median scores
    result = {key: value * 100 for key, value in result.items()}
    return {k: round(v, 4) for k, v in result.items()}

# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# device = "cpu"
# model.to(device)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# print(train_dataset.column_names)
# train_dataset = train_dataset.remove_columns(dataset["train"].column_names)
# feature_dataset = train_dataset.remove_columns(dataset["train"].column_names)
# feature_eval_dataset = eval_dataset.remove_columns(dataset["test"].column_names)

# device = "cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"
# device = "cpu"
# model.to(device)

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [45]:
trainer.train()

# trainer.evaluate()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.820800,0.776124,18.390200,4.611300,12.844200,15.907200
2,0.650500,0.769532,19.184900,4.752000,13.346400,16.301800
3,0.505000,0.810305,19.653100,4.836100,13.481700,16.619200


3000
3000
{'rouge1': 0.18390176782902212, 'rouge2': 0.04611326010842858, 'rougeL': 0.12844173074895218, 'rougeLsum': 0.15907184861506635}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


3000
3000
{'rouge1': 0.1918488203521185, 'rouge2': 0.0475196026742221, 'rougeL': 0.133464237662845, 'rougeLsum': 0.16301767691545285}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


3000
3000
{'rouge1': 0.1965313532970688, 'rouge2': 0.048361289818950645, 'rougeL': 0.13481723150904343, 'rougeLsum': 0.16619222030081854}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=21375, training_loss=0.6587483438413743, metrics={'train_runtime': 7896.6342, 'train_samples_per_second': 21.655, 'train_steps_per_second': 2.707, 'total_flos': 3.70574886961152e+17, 'train_loss': 0.6587483438413743, 'epoch': 3.0})

KeyboardInterrupt: 

In [50]:
prefix = "summarize: "

def tokenize_function(examples):
        # print(examples)
        inputs = [prefix + doc for doc in examples["content"]]
        model_inputs = tokenizer(inputs, padding = "max_length", max_length = 1024, truncation=True)
        summary = tokenizer(examples["summary"], padding = "max_length", max_length=128, truncation=True)
        model_inputs["labels"] = summary["input_ids"]
        return model_inputs
tokenized_train2 = random_train.map(tokenize_function, batched=True)
tokenized_test2 = random_test.map(tokenize_function, batched=True)
train_dataset2 = tokenized_train2
eval_dataset2 = tokenized_test2


In [48]:
model2 = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

In [51]:
trainer2 = Seq2SeqTrainer(
    model2,
    args,
    train_dataset=train_dataset2,
    eval_dataset=eval_dataset2,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [52]:
trainer2.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.842900,0.782388,18.471500,4.631300,12.868100,15.975800
2,0.651300,0.780022,19.275500,4.833000,13.465700,16.439300
3,0.503000,0.814108,19.604000,4.876200,13.531500,16.594400


3000
3000
{'rouge1': 0.18471521175154731, 'rouge2': 0.04631281791468153, 'rougeL': 0.1286806585940681, 'rougeLsum': 0.15975771420342438}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


3000
3000
{'rouge1': 0.19275484327180303, 'rouge2': 0.04833010766598442, 'rougeL': 0.13465747684714113, 'rougeLsum': 0.16439328759059743}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


3000
3000
{'rouge1': 0.196039686238137, 'rouge2': 0.04876182366111146, 'rougeL': 0.13531530441942363, 'rougeLsum': 0.16594364695668987}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=21375, training_loss=0.6657534608004386, metrics={'train_runtime': 7793.0011, 'train_samples_per_second': 21.943, 'train_steps_per_second': 2.743, 'total_flos': 3.70574886961152e+17, 'train_loss': 0.6657534608004386, 'epoch': 3.0})

In [56]:
# trainer2.train(resume_from_checkpoint="model-finetuned/checkpoint-21375")
trainer2.save_model("model-finetuned/model")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


In [57]:
model3 = AutoModelForSeq2SeqLM.from_pretrained("model-finetuned/model")

In [58]:
args2 = Seq2SeqTrainingArguments(
    output_dir=f"model-finetuned",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5.6e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=2,
    predict_with_generate=True,
    logging_steps=logging_steps,
    push_to_hub=False,
    fp16 = True,
    optim="adamw_bnb_8bit"
)
trainer3 = Seq2SeqTrainer(
    model3,
    args2,
    train_dataset=train_dataset2,
    eval_dataset=eval_dataset2,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [60]:
trainer3.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,0.529300,0.855843,19.229600,4.587600,13.236400,16.294700
2,0.377600,0.926419,19.112100,4.472600,13.085000,15.998100


3000
3000
{'rouge1': 0.1922958398402801, 'rouge2': 0.04587564051326931, 'rougeL': 0.13236371328938676, 'rougeLsum': 0.1629465546703732}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


3000
3000
{'rouge1': 0.19112113906984807, 'rouge2': 0.04472595201899306, 'rougeL': 0.13085040117710223, 'rougeLsum': 0.15998112413427662}


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


TrainOutput(global_step=14250, training_loss=0.4534212410910088, metrics={'train_runtime': 5240.5962, 'train_samples_per_second': 21.753, 'train_steps_per_second': 2.719, 'total_flos': 2.640053510476923e+17, 'train_loss': 0.4534212410910088, 'epoch': 2.0})

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [66]:
df = pd.read_csv("drive/MyDrive/Data/all_cleaned_summarization_data.csv")

In [100]:
email = df.loc[15, "text"]
email = "Summarize: " + email
print(email)

Summarize: amazon  --------------------------------------------------------------------------------------------------------------------------------------------    hi [USER],    your package has been delivered!    how was your delivery?  it was great [LINK]  not so great [LINK]    track your package:  [LINK]      ============================================================================================================================================    order info    2 items  order #112-8054629-3347422    return or replace items in your orders:     [LINK]    --------------------------------------------------------------------------------------------------------------------------------------------  this email  was sent from a notification-only email address that cannot accept incoming  email. please do not reply to this message.     © 2019  amazon.com, inc. or its affiliates. all rights reserved. amazon, amazon.com,  and the amazon.com logo are registered trademarks of amazon.com, inc. 

In [70]:
tokenizer = AutoTokenizer.from_pretrained("model-finetuned/checkpoint-21375")
model = AutoModelForSeq2SeqLM.from_pretrained("model-finetuned/checkpoint-21375")

In [101]:
input = tokenizer(email, padding="max_length", truncation=True, max_length = 1024, return_tensors="pt")

In [102]:
outputs = model.generate(**input)

# Decode summaries
generated_summaries = tokenizer.batch_decode(outputs, skip_special_tokens=True)

In [103]:
print(email)
print(generated_summaries)

Summarize: amazon  --------------------------------------------------------------------------------------------------------------------------------------------    hi [USER],    your package has been delivered!    how was your delivery?  it was great [LINK]  not so great [LINK]    track your package:  [LINK]      ============================================================================================================================================    order info    2 items  order #112-8054629-3347422    return or replace items in your orders:     [LINK]    --------------------------------------------------------------------------------------------------------------------------------------------  this email  was sent from a notification-only email address that cannot accept incoming  email. please do not reply to this message.     © 2019  amazon.com, inc. or its affiliates. all rights reserved. amazon, amazon.com,  and the amazon.com logo are registered trademarks of amazon.com, inc. 

Summarize: hi [USER],    i’m checking in on my previous email to you about what is new.    can i set up a call for you?    thank you and i hope you have a wonderful morning,    jessica gold  patron experience assistant  92nd street y  1395 lexington avenue  new york, ny 10128  92ny.org      unsubscribe   [SEP] jessica gold [SEP] jgold@cmail.92y.org [SEP] <USER> [SEP] Updates [SEP] False


TypeError: argument 'ids': Can't extract `str` to `Vec`